<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-1-begin-modeling-process/LS_DS_241_Begin_modeling_process_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working with FMA Free Music Archive data

In [1]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-29 03:30:22--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip.2’

fma_metadata.zip.2  100%[===================>] 341.81M  18.9MB/s    in 19s     

2019-01-29 03:30:42 (17.5 MB/s) - ‘fma_metadata.zip.2’ saved [358412441/358412441]

Archive:  fma_metadata.zip
replace fma_metadata/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
# Import common libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Loading data and setting the column names

tracks = pd.read_csv('fma_metadata/tracks.csv', header=1)
tracks.drop(index=0, inplace=True)
tracks.rename(index=str, columns= {'Unnamed: 0' : 'track_id'}, inplace=True)
tracks.columns

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['track_id', 'comments', 'date_created', 'date_released', 'engineer',
       'favorites', 'id', 'information', 'listens', 'producer', 'tags',
       'title', 'tracks', 'type', 'active_year_begin', 'active_year_end',
       'associated_labels', 'bio', 'comments.1', 'date_created.1',
       'favorites.1', 'id.1', 'latitude', 'location', 'longitude', 'members',
       'name', 'related_projects', 'tags.1', 'website', 'wikipedia_page',
       'split', 'subset', 'bit_rate', 'comments.2', 'composer',
       'date_created.2', 'date_recorded', 'duration', 'favorites.2',
       'genre_top', 'genres', 'genres_all', 'information.1', 'interest',
       'language_code', 'license', 'listens.1', 'lyricist', 'number',
       'publisher', 'tags.2', 'title.1'],
      dtype='object')

In [14]:
print(tracks.shape)

tracks.latitude.fillna(method='ffill', inplace=True)
tracks.longitude.fillna(method='ffill', inplace=True)
tracks.genre_top.fillna(method='ffill', inplace=True)

tracks.isnull().sum()

(106574, 53)


track_id                  0
comments                  0
date_created           3529
date_released         36280
engineer              91279
favorites                 0
id                        0
information           23425
listens                   0
producer              88514
tags                      0
title                  1025
tracks                    0
type                   6508
active_year_begin     83863
active_year_end      101199
associated_labels     92303
bio                   35418
comments.1                0
date_created.1          856
favorites.1               0
id.1                      0
latitude                  0
location              36364
longitude                 0
members               59725
name                      0
related_projects      93422
tags.1                    0
website               27318
wikipedia_page       100993
split                     0
subset                    0
bit_rate                  0
comments.2                0
composer            

In [0]:
tracks.genre_top.isnull().sum()

# Getting sample of data
tracks = tracks.sample(1000, random_state=42)

In [0]:
X = tracks.drop(columns=['date_created',
'date_released',
'engineer',
'information',
'producer',
'active_year_begin',
'active_year_end',
'associated_labels',
'bio',
'location',
'members',
'related_projects',
'website',
'wikipedia_page',
'composer',
'date_recorded',
'information.1',
'language_code',
'lyricist',
'publisher','genre_top'])
X_numeric = tracks.select_dtypes(np.number)
y = tracks.genre_top

In [0]:
# Train/Test Split for numeric data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_numeric, y, test_size=0.2, random_state=42)

In [18]:
# Run Logistic Regression and check accuracy
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(solver='saga', multi_class='multinomial')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.22

Considering only the numerical features the accuracy is 22%

In [19]:
# Using Scalar on numerical columns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(StandardScaler(),
                        LogisticRegression(solver='saga', 
                                           multi_class='multinomial'))

pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)
accuracy_score(y_test, y_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.28

Considering only the numerical features with Scaling the accuracy is 28%

In [20]:
# One-hot-encode all the categorical featues.
!pip install category_encoders

In [0]:
import category_encoders as ce
# Train/Test Split for both numeric and categorical data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)

In [22]:
pipeline = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
                        StandardScaler(),
                        LogisticRegression(solver='saga', 
                                           multi_class='multinomial'))

pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)
accuracy_score(y_test,y_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.595

Considering few categorical features with numerical features with Scaling the accuracy is 60%

In [24]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarn

In [25]:
scores, scores.mean(), scores.std()

(array([0.6091954 , 0.63095238, 0.5       , 0.51219512, 0.64197531,
        0.5       , 0.5       , 0.64102564, 0.54666667, 0.64383562]),
 0.572584613797509,
 0.06280708828072823)

The model has good prediction when compared using Cross validation.